<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/pinn_pipeline_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch numpy matplotlib scikit-learn scipy umap-learn

In [ ]:
# pinn_pipeline.py

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import pearsonr
import umap
import random

# Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)


# 1. Synthetic Data Generators
def generate_dissolution_data(n_samples=500):
    k, C0 = 0.5, 1.0
    t = np.linspace(0, 10, n_samples)[:, None]
    C = C0 * np.exp(-k * t) + 0.01 * np.random.randn(*t.shape)
    return torch.tensor(t, dtype=torch.float32), torch.tensor(C, dtype=torch.float32)


def generate_accumulation_data(n_samples=500):
    k, M0 = 0.3, 0.5
    t = np.linspace(0, 10, n_samples)[:, None]
    M = M0 * np.exp(k * t) + 0.01 * np.random.randn(*t.shape)
    return torch.tensor(t, dtype=torch.float32), torch.tensor(M, dtype=torch.float32)


# 2. PINN Model
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)


# 3. Physics Residuals
def residual_dissolution(model, t):
    t.requires_grad_(True)
    C_pred = model(t)
    dC_dt = torch.autograd.grad(C_pred.sum(), t, create_graph=True)[0]
    return dC_dt + 0.5 * C_pred


def residual_accumulation(model, t):
    t.requires_grad_(True)
    M_pred = model(t)
    dM_dt = torch.autograd.grad(M_pred.sum(), t, create_graph=True)[0]
    return dM_dt - 0.3 * M_pred


# 4. Training Function
def train(model, loader, res_fn, epochs=100, phy_lambda=1.0, lr=1e-3, name="Model"):
    opt = optim.Adam(model.parameters(), lr=lr)
    mse = nn.MSELoss()
    for ep in range(1, epochs + 1):
        dl, pl = 0.0, 0.0
        for t_b, y_b in loader:
            opt.zero_grad()
            y_pred = model(t_b)
            ld = mse(y_pred, y_b)
            res = res_fn(model, t_b)
            lp = mse(res, torch.zeros_like(res))
            (ld + phy_lambda * lp).backward()
            opt.step()
            dl += ld.item()
            pl += lp.item()
        print(f"[{name}] Epoch {ep:3d}/{epochs} — Data: {dl/len(loader):.4f}, Phy: {pl/len(loader):.4f}")
    return model


# 5. MC-Dropout Prediction
def mc_predict(model, t, n_samples=50):
    model.train()
    preds = np.stack([model(t).detach().cpu().numpy() for _ in range(n_samples)], axis=0)
    return preds.mean(axis=0).flatten(), preds.std(axis=0).flatten()


# 6. Main Pipeline
def main():
    # Data
    t_d, C = generate_dissolution_data()
    t_a, M = generate_accumulation_data()
    loader_d = DataLoader(TensorDataset(t_d, C), batch_size=128, shuffle=True)
    loader_a = DataLoader(TensorDataset(t_a, M), batch_size=128, shuffle=True)

    # Train
    md = train(PINN(), loader_d, residual_dissolution, name="DissolutionAI")
    ma = train(PINN(), loader_a, residual_accumulation, name="PreComputationalAI")

    # Inference
    t_full = torch.linspace(0, 10, 500)[:, None]
    mean_d, std_d = mc_predict(md, t_full)
    mean_a, std_a = mc_predict(ma, t_full)
    print("Mean/Std shapes:", mean_d.shape, std_d.shape, mean_a.shape, std_a.shape)

    # Features & UMAP
    def get_feats(m, t):
        x = m.net[0](t); x = torch.tanh(x)
        x = m.net[2](x); x = torch.tanh(x)
        return x.detach().cpu().numpy()

    f_d, f_a = get_feats(md, t_full), get_feats(ma, t_full)
    feats = np.vstack([f_d, f_a])
    emb = umap.UMAP(n_components=2, random_state=SEED).fit_transform(feats)
    print("UMAP shape:", emb.shape)

    # Visualization
    std_all = np.concatenate([std_d, std_a])
    labels = np.array([0]*len(std_d) + [1]*len(std_a))
    plt.figure(figsize=(6, 5))
    plt.scatter(emb[:500,0], emb[:500,1], c='C0', label='Dissolution', alpha=0.6)
    plt.scatter(emb[500:,0], emb[500:,1], c='C1', label='Accumulation', alpha=0.6)
    plt.legend(); plt.title('UMAP Projection'); plt.show()

    # Clustering
    km = KMeans(n_clusters=2, random_state=SEED).fit(emb)
    sil = silhouette_score(emb, km.labels_)
    print("Silhouette:", sil)
    plt.figure(figsize=(6,5))
    plt.scatter(emb[:,0], emb[:,1], c=km.labels_, cmap='tab10', alpha=0.6)
    plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:,1], c='k', marker='X')
    plt.title('KMeans Clusters'); plt.show()

    # Classification
    clf = LogisticRegression().fit(emb, labels)
    print("Classification acc:", clf.score(emb, labels))

    # Uncertainty vs Distance
    dists = np.linalg.norm(emb - km.cluster_centers_[km.labels_], axis=1)
    r, p = pearsonr(dists, std_all)
    print("Unc vs Dist r:", r, "p:", p)
    plt.figure(figsize=(6,4))
    plt.scatter(dists, std_all, alpha=0.6)
    plt.xlabel('Dist to center'); plt.ylabel('Std'); plt.title('Uncertainty vs Dist')
    plt.show()


if __name__ == '__main__':
    main()